In [ ]:
import pandas as pd
import numpy as np
import mayavi as maya
from mayavi import mlab
from functools import partial
mlab.init_notebook('x3d',800,800)

In [ ]:
MZ = 12
MT = 12
def read_w7x(filename):
    R = np.zeros((25, 12))
    Z = np.zeros((25, 12))
    with open(filename) as f:
        _ = f.readline()
        _, Nfp, _ = f.readline().split()
        Nfp = int(Nfp)
        _ = f.readline()
        _ = f.readline()
        for i in range(288):
            n, m, rc, _, _, zs = f.readline().split()
            n = int(n)
            m = int(m)
            rc = float(rc)
            zs = float(zs)
            R[n + 12, m] = rc
            Z[n + 12, m] = zs
    return R, Z, Nfp

def get_xyz(R, Z, zeta, theta, Nfp):
    r = np.zeros((zeta.shape[0], theta.shape[0]))
    z = np.zeros((zeta.shape[0], theta.shape[0]))
    for mz in range(-MZ, MZ + 1):
        for mt in range(MT):
            r += R[mz + MZ, mt] * np.cos( mt * theta[np.newaxis, :] - mz * Nfp * zeta[:, np.newaxis] )
            z += Z[mz + MZ, mt] * np.sin( mt * theta[np.newaxis, :] - mz * Nfp * zeta[:, np.newaxis] )
    x = r * np.cos(zeta)[:, np.newaxis]
    y = r * np.sin(zeta)[:, np.newaxis]
    return np.concatenate((x[:, :, np.newaxis], y[:, :, np.newaxis], z[:, :, np.newaxis]), axis = -1)


def plot_surface(R, Z, NZ, NT, Nfp):
    zeta = np.linspace(0,2 * np.pi, NZ + 1)
    theta = np.linspace(0, 2 * np.pi, NT + 1)
    r = get_xyz(R, Z, zeta, theta, Nfp)
    x = r[:,:,0]
    y = r[:,:,1]
    z = r[:,:,2]
    p = mlab.mesh(x,y,z,color=(0.8,0.0,0.0))
    return p

def compute_drdz(R, Z, zeta, theta, Nfp):
    x = np.zeros((zeta.shape[0], theta.shape[0]))
    y = np.zeros((zeta.shape[0], theta.shape[0]))
    z = np.zeros((zeta.shape[0], theta.shape[0]))
    for mz in range(-MZ, MZ + 1):
        for mt in range(MT):
            coeff = R[mz + MZ, mt]
            z_coeff = Z[mz + MZ, mt]
            arg = mt * theta[np.newaxis, :] - mz * Nfp * zeta[:, np.newaxis]
            x += coeff * np.cos(arg) * (-np.sin(zeta[:, np.newaxis])) + coeff * (mz * Nfp) * np.cos(zeta[:, np.newaxis]) * np.sin(arg)
            y += coeff * np.cos(arg) * np.cos(zeta[:, np.newaxis]) + coeff * (mz * Nfp) * np.sin(arg) * np.sin(zeta[:, np.newaxis])
            z += z_coeff * np.cos(arg) * -(mz * Nfp)
    return np.concatenate((x[:, :, np.newaxis], y[:, :, np.newaxis], z[:, :, np.newaxis]), axis = -1)

def compute_drdt(R, Z, zeta, theta, Nfp):
    x = np.zeros((zeta.shape[0], theta.shape[0]))
    y = np.zeros((zeta.shape[0], theta.shape[0]))
    z = np.zeros((zeta.shape[0], theta.shape[0]))
    for mz in range(-MZ, MZ + 1):
        for mt in range(MT):
            coeff = R[mz + MZ, mt]
            z_coeff = Z[mz + MZ, mt]
            arg = mt * theta[np.newaxis, :] - mz * Nfp * zeta[:, np.newaxis]
            x += coeff * np.sin(arg) * -mt * np.cos(zeta[:, np.newaxis])
            y += coeff * np.sin(arg) * -mt * np.sin(zeta[:, np.newaxis])
            z += z_coeff * np.cos(arg) * mt
    return np.concatenate((x[:, :, np.newaxis], y[:, :, np.newaxis], z[:, :, np.newaxis]), axis = -1)

def get_w7x_data(R, Z, NZ, NT, Nfp):
    zeta = np.linspace(0,2 * np.pi, NZ + 1)
    theta = np.linspace(0, 2 * np.pi, NT + 1)
    r = get_xyz(R, Z, zeta, theta, Nfp)
    drdz = compute_drdz(R, Z, zeta, theta, Nfp)
    drdt = compute_drdt(R, Z, zeta, theta, Nfp)
    N = np.cross(drdz, drdt)
    sg = np.linalg.norm(N, axis=-1)
    nn = N / sg[:,:,np.newaxis]
    return r, nn, sg
    
def plot_coils(r_coils):
    for ic in range(r_coils.shape[0]):
        p = mlab.plot3d(r_coils[ic,:,0], r_coils[ic,:,1], r_coils[ic,:,2], tube_radius=0.04, color = (0.0, 0.0, 0.8))#, line_width = 0.01,)
    return p
    

In [ ]:
R, Z, Nfp = read_w7x("../../focusadd/initFiles/w7x/plasma.boundary_std")

In [ ]:
mlab.clf()
p = plot_surface(R, Z, 128, 32, Nfp)
p

In [ ]:
r, nn, sg = get_w7x_data(R, Z, 5 * 30, 16, Nfp)

In [ ]:
mlab.clf()
x = r[:,:,0]
y = r[:,:,1]
z = r[:,:,2]
p = mlab.mesh(x,y,z,color=(0.8,0.0,0.0))
nn_x = nn[:,:,0]
nn_y = nn[:,:,1]
nn_z = nn[:,:,2]
s = mlab.quiver3d(x[::,::],y[::,::],z[::,::],sg*nn_x[::,::],sg*nn_y[::,::],sg*nn_z[::,::])
s

In [ ]:
def read_coils(filename):
    r = np.zeros((50, 96, 3))
    with open(filename) as f:
        _ = f.readline()
        _ = f.readline()
        _ = f.readline()
        for i in range(50):
            for s in range(96):
                x, y, z, _ = f.readline().split()
                r[i, s, 0] = float(x)
                r[i, s, 1] = float(y)
                r[i, s, 2] = float(z)
            _ = f.readline()
    return r

In [ ]:
r_c = read_coils("../../focusadd/initFiles/w7x/coils.std_mc")

In [ ]:
p = plot_coils(r_c)
p

In [ ]:
def compute_coil_fourierSeries(NF, r_centroid):
        NC = r_centroid.shape[0]
        NS = r_centroid.shape[1]
        x = r_centroid[:, :, 0]
        y = r_centroid[:, :, 1]
        z = r_centroid[:, :, 2]
        xc = np.zeros((NC, NF))
        yc = np.zeros((NC, NF))
        zc = np.zeros((NC, NF))
        xs = np.zeros((NC, NF))
        ys = np.zeros((NC, NF))
        zs = np.zeros((NC, NF))
        xc[:,0] += np.sum(x, axis=1) / NS
        yc[:,0] += np.sum(y, axis=1) / NS
        zc[:,0] += np.sum(z, axis=1) / NS
        theta = np.linspace(0, 2 * np.pi, NS + 1)[0:NS]
        for m in range(1, NF):
            xc[:, m] += 2.0 * np.sum(x * np.cos(m * theta), axis=1) / NS
            yc[:, m] += 2.0 * np.sum(y * np.cos(m * theta), axis=1) / NS
            zc[:, m] += 2.0 * np.sum(z * np.cos(m * theta), axis=1) / NS
            xs[:, m] += 2.0 * np.sum(x * np.sin(m * theta), axis=1) / NS
            ys[:, m] += 2.0 * np.sum(y * np.sin(m * theta), axis=1) / NS
            zs[:, m] += 2.0 * np.sum(z * np.sin(m * theta), axis=1) / NS
        return np.asarray([xc, yc, zc, xs, ys, zs])  # 6 x NC x NF

In [ ]:
fc = compute_coil_fourierSeries(10, r_c)
print(fc.shape)

In [ ]:
    def compute_r_centroid(NS, fc):
        theta = np.linspace(0, 2 * np.pi, NS + 1)
        NC = fc.shape[1]
        NF = fc.shape[2]
        xc = fc[0]
        yc = fc[1]
        zc = fc[2]
        xs = fc[3]
        ys = fc[4]
        zs = fc[5]
        x = np.zeros((NC, NS + 1))
        y = np.zeros((NC, NS + 1))
        z = np.zeros((NC, NS + 1))
        for m in range(NF):
            arg = m * theta
            carg = np.cos(arg)
            sarg = np.sin(arg)
            x += (
                xc[:, np.newaxis, m] * carg[np.newaxis, :]
                + xs[:, np.newaxis, m] * sarg[np.newaxis, :]
            )
            y += (
                yc[:, np.newaxis, m] * carg[np.newaxis, :]
                + ys[:, np.newaxis, m] * sarg[np.newaxis, :]
            )
            z += (
                zc[:, np.newaxis, m] * carg[np.newaxis, :]
                + zs[:, np.newaxis, m] * sarg[np.newaxis, :]
            )
        return np.concatenate(
            (x[:, :, np.newaxis], y[:, :, np.newaxis], z[:, :, np.newaxis]), axis=2
        )

In [ ]:
r_c_2 = compute_r_centroid(96, fc)

In [ ]:
mlab.clf()
p = plot_surface(R, Z, 128, 32, Nfp)
p = plot_coils(r_c_2)
p

In [ ]:
with open('w7x_r_surf.npy', 'wb') as f:
    np.save(f, r)
with open('w7x_nn_surf.npy', 'wb') as f:
    np.save(f, nn)
with open('w7x_sg_surf.npy', 'wb') as f:
    np.save(f, sg)
with open('w7x_fc.npy', 'wb') as f:
    np.save(f, fc)